# BreCaNet
Breast Cancer Network

How to get the data:
Came from the TCGA database (https://portal.gdc.cancer.gov/repository)

##### Gene Expression Data
On the "Cases" tab, select "breast" in "Primary site," "ductal and lobular neoplasms" in "Disease type," "female" in "Gender," and "White" in "Race"

On the "Files" tab, "RNA-seq" in "Experimental Strategy," and "HTSeq-Counts" in "Workflow" 

This results 714 cases and 832 total files. Click on the "Download" button and select "Cart"

##### Clinical Data with IHC Receptor Status
Using the same TCGA repository, on the "Cases" tab, make sure to use the same settings as the gene expression data

On the "Files" tab, select "Clinical" in "Data Category," "BCR Biotab" in "Data Format"

Download the "nationwidechildrens.org_clinical_patient_brca.txt" file

##### TF Binding Motif Data
Data retrieved from the Glass lab (https://sites.google.com/a/channing.harvard.edu/kimberlyglass/home)

To get the data, go to "Tools and Resources" then "Motif Mappings"

Next to the "Human Motif Scan" heading, click on "Motif Info" for the data

##### PPI Data

In [12]:
# Necessary to import

import os
import gzip
import shutil
import numpy as np

### Testing

The first 5 files from the complete gene expression data set were selected to edit the code
"sampleData" is a folder containg the unzipped data
"sampleDataRaw" is a folder with unedited zipped data

In [13]:
# User's paths

samplePath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/sampleData'
sampleNewFolder = '/Users/ursulawidocki/Desktop/BreCaNet/Data/sampleDataRaw'
clinicalPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/clinicalData/nationwidechildrens.org_clinical_patient_brca.txt'

In [14]:
for fileName in os.listdir(samplePath):

    #print(fileName)
    if fileName == '.DS_Store'or fileName == 'MANIFEST.txt':
            #print(' SKIPPED: ', fileName)
            continue
    newPath = samplePath +'/'+ fileName
    #print('PATH: ', newPath)
    
    # goes into the individual folder
    for indivName in os.listdir(newPath):
        
        if (indivName == '.DS_Store') or (indivName == 'MANIFEST.txt'):
            print(' DATA SKIPPED: ', indivName)
            continue
        dataPath = newPath + '/' + indivName
        #print('DATA PATH: ', dataPath)
    
        fileEnding = indivName[-3] + indivName[-2] + indivName[-1]
        if(fileEnding == '.gz'):
            with gzip.open(dataPath, 'rb') as f_in:
                newFilePath = sampleNewFolder + '/' + indivName[0:-3]
                #print("OG: ", dataPath)
                #print("NEW: ", newFilePath)
                with open(newFilePath, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
                    
                    
        #extracts clinical information based on fileName
        with open(clinicalPath, "r") as f_clinic:
            for clinic_line in f_clinic.readlines():
                Name = fileName #by fileName
                #Name = indivName[0:-16] #by indivName
                #print(Name)
                clinic = clinic_line.strip().split("\t")[0].lower()
                
                if(Name == clinic):
                    print("Yay")
                    

 DATA SKIPPED:  .DS_Store


Gets list of genes and patient names

In [7]:
path = sampleNewFolder
geneList = list()
indivList = list()
file1 = 0

for fileName in os.listdir(path):
    if fileName == '.DS_Store'or fileName == 'MANIFEST.txt':
            #print(' SKIPPED: ', fileName)
            continue
    readFile = path + '/' + fileName
    
    with open(readFile, "r") as file:
            
        for line in file.readlines():
            if file1 == 1:
                continue
            #geneSet.add(line.strip().split("\t")[0])
            geneList.append(line.strip().split("\t")[0])
    
    indivList.append(fileName[0:-16])
    
    file1 = 1      
    
   
print(len(geneList))
print(len(indivList))

60488
5


Makes a matrix where the rows are genes and the columns are patients

In [11]:
allTesting = np.zeros((len(geneList), len(indivList)))
j = 0
i = 0

for fileName in os.listdir(path):
    if fileName == '.DS_Store'or fileName == 'MANIFEST.txt':
            #print(' SKIPPED: ', fileName)
            continue
    readFile = path + '/' + fileName
    
    with open(readFile, "r") as file:
        for line in file.readlines():
            allTesting[i,j] = line.strip().split("\t")[1]
            
            if i < len(geneList):
                i+=1
            
    j+=1
    i = 0

In [18]:
## TESTING: 3
## views clinical data

clinicalPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/clinicalData/nationwidechildrens.org_clinical_patient_brca.txt'
# has 1700943 lines -> statinfo = os.stat(clinicalPath); print(statinfo.st_size)

# column 0 is patient id, which should align with the file/folder name
# column 1 is patient barcode, I dont think I need that
# column 8 is patient race
# column 12 tumor status
# column 18 histologic diagnosis
# column 25 margin status
# column 28 surgery margin status

# column 43 estrogen receptor status by ihc
# column 46 immunohistochemistry score for er
# column 49 progesterone receptor status by ihc
# column 52 immunohistochemistry score for pr
# column 55 HER2 status by ihc
# column 57 immunohistochemistry score for HER2

# column 100 histology type (ex: infiltrating lobular carcinoma, infiltrating ductal carcinoma, other, mixed)
# column 111 last column

with open(clinicalPath, "r") as clinicFile:
    for line in clinicFile.readlines():
        print(line.strip().split("\t")[0])

bcr_patient_uuid
bcr_patient_uuid
CDE_ID:
6E7D5EC6-A469-467C-B748-237353C23416
55262FCB-1B01-4480-B322-36570430C917
427D0648-3F77-4FFC-B52C-89855426D647
C31900A4-5DCD-4022-97AC-638E86E889E4
6623FC5E-00BE-4476-967A-CBD55F676EA6
86C6F993-327F-4525-9983-29C55625593A
16FC3677-0393-4ED1-AD3F-C8355F056369
2FD36838-5A83-433E-AC80-B1F77448E5AA
0045349c-69d9-4306-a403-c9c1fa836644
c462e422-eb8d-4daf-9897-2a9c6cbd783a
c397e6c2-ba3a-4f5e-a258-c140611192fe
4b620966-b4c7-4615-bf58-ca9d3226b9bd
0ed9ad26-7fdc-4d75-93f9-d029eee94993
a6a863d9-b39d-4965-b47f-79a1bcf3c976
16368c32-2118-4fcf-8693-6c89995e49d8
a2db9dd1-44d5-48b5-817c-a21a85fadb21
a5b44d66-c162-46b5-9df2-86305f0385c5
784de7ac-8424-42eb-83d4-a1bebaa42b97
0dc337fa-da8b-42c4-b9a7-fb76d81c161f
6644fd4e-d2fe-4785-a73c-0f36fcc740e2
a9bb8159-32f0-454c-a946-b3286a52b9d5
08de63a2-7b76-43c3-80dc-df748b1d81bc
197428f7-bd77-4c5f-8119-87157b5c35b9
ccd4a24b-d8cc-4686-9dee-c98b0c5a8d21
b5fba77b-1f50-4e71-95b2-566afba4bdd7
f3a058b9-fb7f-4098-8f0e-83c2acfa1

In [ ]:
## TESTING: 4
## reads in the TF-gene data from Glass lab scanner and extracts data necessary for genes present

motifPath = '/Users/ursulawidocki/Desktop/BreCaNet/Data/Homo_sapiens_motifinfo.txt'

with open(motifPath, "r") as file:
    next(file) #skips the first line since that is just a header
    for line in file.readlines():
        print(line.strip().split("\t")[1])


## once I get the clinical data from TCGA, I need to match the individual
## -> so find the intersect of their genes

## from the TCGA site, it says that there are 717 clinical data
## and 3,562 biospecimen

## combine into one matrix

## 


## try regular sets

In [ ]:
## normalize the genes counts using TPM

In [ ]:
## TESTING:
## PPI data from Glass